In [ ]:
# Importe as bibliotecas necessárias
import pandas as pd
import holidays
import numpy as np
import matplotlib.pyplot as plt
import optuna
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.metrics import mean_squared_error, mean_absolute_error

from prophet import Prophet

In [ ]:
# Carregue os dados
data = pd.read_csv('../TCC/datasets/demanda.csv')
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data = data.set_index('Timestamp').asfreq('H')

# Defina a coluna alvo
target = 'kVA fornecido'

In [ ]:
# Removendo a coluna 'Dia' 
data = data.drop(['Dia','Holiday'], axis=1)

# Ajustando Posto horários para inteiro
data['Postos horários'] = data['Postos horários'].astype(int)

In [ ]:
# Criando as colunas de data e hora
data['Hour'] = data.index.hour
data['Day'] = data.index.day
data['Dayofweek'] = data.index.dayofweek
data['Month'] = data.index.month
data['Quarter'] = data.index.quarter
data['Year'] = data.index.year
# Criando a coluna 'Holiday' com os feriados brasileiros
br_holidays = holidays.Brazil(years=data.index.year.unique())
data['Holiday'] = data.index.map(lambda x: x in br_holidays).astype(int)

In [ ]:
# Movendo os últimos 24 dados (1 dia) para serem usados como dados de validação da previsão
df_forecast = data.iloc[-24:]
data = data.iloc[:-24]

In [ ]:
columns = ['kVA fornecido', 'kW fornecido', 'kvar indutivo']

f, axes = plt.subplots(nrows=3, ncols=1, figsize=(18, 10))
f.suptitle('Potência média diária', weight='bold', fontsize=18)

for i, col in enumerate(columns):
    sns.boxplot(data=data, x='Hour', y=col, ax=axes.flatten()[i], palette='Set1')

In [ ]:
columns = ['kVA fornecido', 'kW fornecido', 'kvar indutivo']

f, axes = plt.subplots(nrows=3, ncols=1, figsize=(18, 10))
f.suptitle('Potência média mensal', weight='bold', fontsize=18)

for i, col in enumerate(columns):
    sns.stripplot(data=data, x='Day', y=col, ax=axes.flatten()[i], palette='Set1')

In [ ]:
columns = ['kVA fornecido', 'kW fornecido', 'kvar indutivo']

f, axes = plt.subplots(nrows=3, ncols=1, figsize=(18, 10))
f.suptitle('Potência média anual', weight='bold', fontsize=18)

for i, col in enumerate(columns):
    sns.barplot(data=data, x='Month', y=col, ax=axes.flatten()[i], palette='Set1')

In [ ]:
columns = ['kVA fornecido', 'kW fornecido', 'kvar indutivo']

f, axes = plt.subplots(nrows=3, ncols=1, figsize=(18, 10))
f.suptitle('Potência média anual', weight='bold', fontsize=18)

for i, col in enumerate(columns):
    sns.violinplot(data=data, x='Year', y=col, ax=axes.flatten()[i], palette='Set1')

In [ ]:
# Plotando a decomposição da série temporal
plt.rcParams['figure.figsize'] = (18, 10)
seasonal_decompose(data[target], period=365).plot()
plt.show()

In [ ]:
# Before building and training our model, let's split the data into training and testing
split_on = '2023-02-01'
df_train, df_test = data[data.index < split_on], data[data.index >= split_on]
# Split the training data into training and validation, with 20% of the training data set aside for validation
valid_split_on = len(df_train) - int(len(df_train) * 0.2)
df_train, df_valid = df_train[:valid_split_on], df_train[valid_split_on:]

print('Train:\t', len(df_train))
print('Valid:\t', len(df_valid))
print('Test:\t', len(df_test))
print('Forecast:\t', len(df_forecast))

In [ ]:
plt.figure(figsize=(20,8))
df_train[target].plot(label='Training Set')
plt.axvline(df_train.index.min(), color='black', ls='--', lw=3)
df_valid[target].plot(label='Validation Set')
plt.axvline(df_train.index.max(), color='black', ls='--', lw=3)
df_test[target].plot(label='Test Set')
plt.axvline(split_on, color='black', ls='--', lw=3)
df_forecast[target].plot(label='Forecast Set')
plt.axvline(df_test.index.max(), color='black', ls='--', lw=3)
plt.title('Data Splitting', weight='bold', fontsize=25)
plt.legend()

# Prophet

In [ ]:
def index_to_column(data):
    data = data.reset_index()
    data['Timestamp'] = pd.to_datetime(data['Timestamp'])
    data = data.sort_values('Timestamp')
    data = data.rename(columns={'Timestamp': 'ds', target: 'y'})
    data = data[['ds', 'y']]
    return data

In [ ]:
prophet_train = index_to_column(df_train)
prophet_valid = index_to_column(df_valid)
prophet_test = index_to_column(df_test)
prophet_forecast = index_to_column(df_forecast)

In [ ]:
# Drop all columns except ds and y
prophet_train = prophet_train[['ds', 'y']]
prophet_valid = prophet_valid[['ds', 'y']]
prophet_test = prophet_test[['ds', 'y']]
prophet_forecast = prophet_forecast[['ds', 'y']]

In [ ]:
prophet_model = Prophet(interval_width=0.95)
prophet_model.fit(pd.concat([prophet_train, prophet_valid]))
prophet_pred = prophet_model.predict(prophet_test[['ds']]) # Keep the dataset format

In [ ]:
mae = round(mean_absolute_error(prophet_test['y'], prophet_pred['yhat']), 3)

In [ ]:
# Crie a figura usando Plotly
fig = go.Figure()

# Adicione as linhas ao gráfico
fig.add_trace(go.Scatter(x=prophet_test['ds'], y=prophet_test['y'], mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=prophet_pred['ds'], y=prophet_pred['yhat'], mode='lines', name='Predicted'))

# Atualize o layout do gráfico
fig.update_layout(
    title=f'Test Forecasting: {mae}',
    title_font=dict(size=25),
    xaxis=dict(title='X Axis Label'),
    yaxis=dict(title='Y Axis Label'),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
)

# Exiba o gráfico
fig.show()

<h1 style="font-family: Trebuchet MS; font-size: 20px; color: #da627d; text-align: left; "><b>Predict the value for next day</b></h1>

In [ ]:
# This time, we will use all data (train and test) to train our model
new_df = index_to_column(data)
prophet_model2 = Prophet(interval_width=0.95)
prophet_model2.fit(new_df)

In [ ]:
# 1 day to the future (1x24 = 24)
future_dates = prophet_model2.make_future_dataframe(periods=24, freq='H')
prophet_pred2 = prophet_model2.predict(future_dates)

In [ ]:
# Drop all lines before the first validation date
prophet_pred2 = prophet_pred2[prophet_pred2['ds'] >= df_forecast.index[0]]
# Reset index to start from 0
prophet_pred2 = prophet_pred2.reset_index(drop=True)

In [ ]:
# Crie a figura usando Plotly
fig = go.Figure()

# Adicione as linhas ao gráfico
fig.add_trace(go.Scatter(x=prophet_pred2['ds'], y=prophet_valid['y'], mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=prophet_pred2['ds'], y=prophet_pred2['yhat'], mode='lines', name='Predicted'))
fig.add_trace(go.Scatter(x=prophet_pred2['ds'], y=prophet_pred2['yhat_upper'], mode='lines', name='Upper Bound', line=dict(color='rgba(0,0,0,0)')))
fig.add_trace(go.Scatter(x=prophet_pred2['ds'], y=prophet_pred2['yhat_lower'], mode='lines', name='Lower Bound', fill='tonexty', line=dict(color='rgba(0,0,0,0)')))

# Atualize o layout do gráfico
fig.update_layout(
    title='Test Forecasting',
    title_font=dict(size=25),
    xaxis=dict(title='X Axis Label'),
    yaxis=dict(title='Y Axis Label'),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
)

# Exiba o gráfico
fig.show()

In [ ]:
prophet_model.plot_components(prophet_pred2)
plt.show()

# Optuna

In [ ]:
import logging
prophet_train['cap'] = data[target].max()
prophet_train['floor'] = data[target].min()
prophet_valid['cap'] = data[target].max()
prophet_valid['floor'] = data[target].min()


def objective(trial):
    params = {
        'changepoint_prior_scale': trial.suggest_float('changepoint_prior_scale', 0.005, 5),
        'changepoint_range': trial.suggest_float('changepoint_range', 0.8, 0.9),
        'seasonality_prior_scale': trial.suggest_float('seasonality_prior_scale', 0.1, 10),
        'holidays_prior_scale': trial.suggest_float('holidays_prior_scale', 0.1, 10),
        'seasonality_mode': trial.suggest_categorical('seasonality_mode', ['multiplicative', 'additive']),
        'growth': trial.suggest_categorical('growth', ['linear', 'logistic']),
        'weekly_seasonality': trial.suggest_int('weekly_seasonality', 5, 10),
        'yearly_seasonality': trial.suggest_int('yearly_seasonality', 1, 20)
    }
    logging.getLogger("cmdstanpy").disabled = True #  turn 'cmdstanpy' logs off
    m = Prophet(**params)
    m.add_country_holidays(country_name='BR')
    m.fit(prophet_train)
    logging.getLogger("cmdstanpy").disabled = False #  revert original setting
    preds = m.predict(prophet_valid[['ds', 'cap', 'floor']])
    
    mae_score = mean_absolute_error(prophet_valid['y'], preds['yhat'])
    return mae_score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

fig = optuna.visualization.plot_optimization_history(study)
fig.show()

fig = optuna.visualization.plot_timeline(study)
fig.show()


In [ ]:
# Obtenha os melhores hiperparâmetros encontrados
best_params = study.best_params

In [ ]:
prophet_test['cap'] = data[target].max()
prophet_test['floor'] = data[target].min()
prophet_model = Prophet(interval_width=0.95, **best_params)
prophet_model.fit(pd.concat([prophet_train, prophet_valid]))
prophet_pred = prophet_model.predict(prophet_test[['ds', 'cap', 'floor']]) # Keep the dataset format

In [ ]:
mae = round(mean_absolute_error(prophet_test['y'], prophet_pred['yhat']), 3)

In [ ]:
# Crie a figura usando Plotly
fig = go.Figure()

# Adicione as linhas ao gráfico
fig.add_trace(go.Scatter(x=prophet_test['ds'], y=prophet_test['y'], mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=prophet_pred['ds'], y=prophet_pred['yhat'], mode='lines', name='Predicted'))

# Atualize o layout do gráfico
fig.update_layout(
    title=f'Test Forecasting: {mae}',
    title_font=dict(size=25),
    xaxis=dict(title='X Axis Label'),
    yaxis=dict(title='Y Axis Label'),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
)

# Exiba o gráfico
fig.show()

<h1 style="font-family: Trebuchet MS; font-size: 20px; color: #da627d; text-align: left; "><b>Predict the value for next day</b></h1>

In [ ]:
# This time, we will use all data (train and test) to train our model
new_df = index_to_column(data)
prophet_model2 = Prophet(interval_width=0.95)
prophet_model2.fit(new_df)

In [ ]:
# 1 day to the future (1x24 = 24)
future_dates = prophet_model2.make_future_dataframe(periods=24, freq='H')
prophet_pred2 = prophet_model2.predict(future_dates)

In [ ]:
# Drop all lines before the first validation date
prophet_pred2 = prophet_pred2[prophet_pred2['ds'] >= df_forecast.index[0]]
# Reset index to start from 0
prophet_pred2 = prophet_pred2.reset_index(drop=True)

In [ ]:
# Crie a figura usando Plotly
fig = go.Figure()

# Adicione as linhas ao gráfico
fig.add_trace(go.Scatter(x=prophet_pred2['ds'], y=prophet_valid['y'], mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=prophet_pred2['ds'], y=prophet_pred2['yhat'], mode='lines', name='Predicted'))
fig.add_trace(go.Scatter(x=prophet_pred2['ds'], y=prophet_pred2['yhat_upper'], mode='lines', name='Upper Bound', line=dict(color='rgba(0,0,0,0)')))
fig.add_trace(go.Scatter(x=prophet_pred2['ds'], y=prophet_pred2['yhat_lower'], mode='lines', name='Lower Bound', fill='tonexty', line=dict(color='rgba(0,0,0,0)')))

# Atualize o layout do gráfico
fig.update_layout(
    title='Test Forecasting',
    title_font=dict(size=25),
    xaxis=dict(title='X Axis Label'),
    yaxis=dict(title='Y Axis Label'),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
)

# Exiba o gráfico
fig.show()

In [ ]:
prophet_model.plot_components(prophet_pred2)
plt.show()